In [1]:
#conv Neural Network
# tensorboard --logdir=/home/ncc/notebook/learn/tensorboard/log
"""
created by kim Seong jung

"""
import numpy as np 
import tensorflow as tf
import re

import math
import time
import matplotlib.pyplot as plt
%matplotlib inline
import os 

file_locate='./cifar_merge/'
sess = tf.InteractiveSession()
test_img=np.load(file_locate+'test_img.npy');
try:
    print np.shape(test_img)
    img_row = np.shape(test_img)[1]
    img_col = np.shape(test_img)[2]
except:
    np.shape(test_img)
    test_img=np.reshape(test_img , newshape = [np.shape(test_img)[0] , 32, 32 ,3] )
    img_row = np.shape(test_img)[1]
    img_col = np.shape(test_img)[2]

    
divide_flag= False
batch_size=30
print img_row ,img_col
n_classes =10
in_ch =3
out_ch1=200
out_ch2=200
out_ch3=200
out_ch4=200
out_ch5=200


fully_ch1=1024
fully_ch2 =1024
fully_ch3 =1024



strides_1=[1,2,2,1]
strides_2=[1,1,1,1]
strides_3=[1,1,1,1]
strides_4=[1,1,1,1]
strides_5=[1,1,1,1]


x= tf.placeholder("float",shape=[None,img_col , img_row ,3],  name = 'x-input')
y_=tf.placeholder("float",shape=[None , n_classes] , name = 'y-input')
keep_prob = tf.placeholder("float")

x_image= tf.reshape(x,[-1,img_row,img_col,3])

iterate=1000




weight_row =3 ; weight_col=3

pooling_row_size1=int(img_row/2)
pooling_row_size2=int(pooling_row_size1/2)
pooling_row_size3=int(pooling_row_size2/2)
pooling_row_size4=int(pooling_row_size3/2)
pooling_row_size5=int(pooling_row_size4/2)
pooling_col_size1=int(img_col/2)
pooling_col_size2=int(pooling_col_size1/2)
pooling_col_size3=int(pooling_col_size2/2)
pooling_col_size4=int(pooling_col_size3/2)
pooling_col_size5=int(pooling_col_size4/2)

print img_col , img_row

(5000, 32, 32, 3)
32 32
32 32


In [2]:
import os 
os.getcwd()

'/home/user01/notebook'

In [3]:
with tf.device('/gpu:3'):
#with tf.device('/gpu:1'):

    if divide_flag == False:
        train_img=np.load(file_locate+'train_img.npy');
        train_lab=np.load(file_locate+'train_lab.npy');
        val_img= np.load(file_locate+'val_img.npy');
        val_lab = np.load(file_locate+'val_lab.npy');
        test_img=np.load(file_locate+'test_img.npy');
        test_lab=np.load(file_locate+'test_lab.npy');

        print "Training Data",np.shape(train_img)
        print "Training Data Label",np.shape(train_lab)
        print "Test Data Label",np.shape(test_lab)
        print "val Data Label" , np.shape(val_img)

        n_train= np.shape(train_img)[0]
        n_train_lab = np.shape(train_lab)[0]

    if divide_flag == True:
        train_img=np.load(file_locate+'train_img_1.npy');
        train_lab=np.load(file_locate+'train_lab_1.npy');
        val_img= np.load(file_locate+'val_img.npy');
        val_lab = np.load(file_locate+'val_lab.npy');
        test_img=np.load(file_locate+'test_img.npy');
        test_lab=np.load(file_locate+'test_lab.npy');

        print "Training Data",np.shape(train_img)
        print "Training Data Label",np.shape(train_lab)
        print "Test Data Label",np.shape(test_lab)
        print "val Data Label" , np.shape(val_lab)

        n_train= np.shape(train_img)[0]
        n_train_lab = np.shape(train_lab)[0]


Training Data (50000, 32, 32, 3)
Training Data Label (50000, 10)
Test Data Label (5000, 10)
val Data Label (5000, 32, 32, 3)


In [4]:
"""def weight_variable(name,shape):
    #initial = tf.truncated_normal(shape , stddev=0.1)
    initial = tf.get_variable(name,shape=shape , initializer = tf.contrib.layers.xavier_initializer())
    return tf.Variable(initial)"""
with tf.device('/gpu:0'):
    def bias_variable(shape , name):
        initial = tf.constant(0.1 , shape=shape)
        return tf.Variable(initial)



In [5]:
with tf.device('/gpu:0'):
    def next_batch(batch_size , image , label):

        a=np.random.randint(np.shape(image)[0] -batch_size)
        batch_x = image[a:a+batch_size,:]
        batch_y= label[a:a+batch_size,:]
        return batch_x, batch_y


In [6]:
with tf.device('/gpu:0'):

    def conv2d(x,w,strides_ , name):
        return tf.nn.conv2d(x,w, strides = strides_, padding='SAME' , name=name )
    def max_pool_2x2(x , name):
        return tf.nn.max_pool(x , ksize=[1,2,2,1] ,strides = [1,2,2,1] , padding = 'SAME' , name=name  )

In [7]:
with tf.variable_scope("layer1") as scope:
    try:
        w_conv1 = tf.get_variable("W1",[weight_row,weight_col,in_ch,out_ch1] , initializer = tf.contrib.layers.xavier_initializer())
    except:
        scope.reuse_variables()
        w_conv1 = tf.get_variable("W1",[weight_row,weight_col,in_ch,out_ch1] , initializer = tf.contrib.layers.xavier_initializer())
with tf.variable_scope("layer1") as scope:
    try:
        b_conv1 = bias_variable([out_ch1] , name ='B1')
    except:
        scope.reuse_variables()
        b_conv1 = bias_variable([out_ch1],name ='B1')
                
            
            
with tf.variable_scope('layer2') as scope:
    try:
        w_conv2 = tf.get_variable("W2",[weight_row,weight_col,out_ch1,out_ch2] , initializer = tf.contrib.layers.xavier_initializer())
    except:
        scope.reuse_variables()
        w_conv2 = tf.get_variable("W2",[weight_row,weight_col,out_ch1,out_ch2] , initializer = tf.contrib.layers.xavier_initializer())
        
with tf.variable_scope('layer2') as scope:
    try:
        b_conv2= bias_variable([out_ch2],name ='B2')
    except:
        scope.reuse_variables()
        b_conv2= bias_variable([out_ch2],name ='B2')
                
with tf.variable_scope('layer3') as scope:
    try:
        w_conv3 = tf.get_variable("W3" ,[weight_row,weight_col,out_ch2,out_ch3] , initializer = tf.contrib.layers.xavier_initializer())
    except:
        scope.reuse_variables()
        w_conv3 = tf.get_variable("W3" ,[weight_row,weight_col,out_ch2,out_ch3] , initializer = tf.contrib.layers.xavier_initializer())
with tf.variable_scope('layer3') as scope:
    try:
        b_conv3 = bias_variable([out_ch3],name ='B3')
    except:
        scope.reuse_variables()
        b_conv3 = bias_variable([out_ch3],name ='B3')
        
with tf.variable_scope('layer4') as scope:
    try:
        w_conv4 =tf.get_variable("W4" ,[weight_row,weight_col,out_ch3,out_ch4] , initializer = tf.contrib.layers.xavier_initializer())
    except:
        scope.reuse_variables()
        w_conv4 = tf.get_variable("W4" ,[weight_row,weight_col,out_ch3,out_ch4] , initializer = tf.contrib.layers.xavier_initializer())
with tf.variable_scope('layer4') as scope:
    try:
        b_conv4 = bias_variable([out_ch4],name ='B4')
    except:
        scope.reuse_variables()
        b_conv4 = bias_variable([out_ch4],name ='B4')
        
with tf.variable_scope('layer5') as scope:
    try:
        w_conv5 = tf.get_variable("W5",[weight_row,weight_col,out_ch4,out_ch5] , initializer = tf.contrib.layers.xavier_initializer())
    except:
        scope.reuse_variables()
        w_conv5 = tf.get_variable("W5" ,[weight_row,weight_col,out_ch4,out_ch5] , initializer = tf.contrib.layers.xavier_initializer())
with tf.variable_scope('layer5') as scope:
    try:
        b_conv5 = bias_variable([out_ch5],name ='B5')
    except:
        scope.reuse_variables()
        b_conv5 = bias_variable([out_ch5],name ='B5')
                

In [8]:
#conncect hidden layer 
with tf.device('/gpu:0'):
    with tf.variable_scope("convolution_1") as scope:
        h_conv1 = tf.nn.relu(conv2d(x_image , w_conv1 ,strides_1 , 'x_image_wconv1')+b_conv1)
    with tf.variable_scope("convolution_2") as scope:
        h_conv2 = tf.nn.relu(conv2d(h_conv1 , w_conv2 ,strides_2 , 'conv1_conv2')+b_conv2)
        h_conv2 = max_pool_2x2(h_conv2 , 'conv2_maxpool')#pooling
    with tf.variable_scope("convolution_3") as scope:
        h_conv3 = tf.nn.relu(conv2d(h_conv2 , w_conv3,strides_3,'conv2_conv3')+b_conv3)
    with tf.variable_scope("convolution_4") as scope:
        h_conv4 = tf.nn.relu(conv2d(h_conv3 , w_conv4,strides_4,'conv3_conv4')+b_conv4)
        h_pool4 = max_pool_2x2(h_conv4,'conv4_maxpool') #pooling 
    with tf.variable_scope("convolution_5") as scope:
        h_conv5 = tf.nn.relu(conv2d(h_conv4, w_conv5,strides_5 ,'conv4_conv5')+b_conv5)
        h_conv5= max_pool_2x2(h_conv5 , 'conv5_maxpool') #pooling 

    print h_conv1
    print h_conv2
    print h_conv3
    print h_conv4
    print h_conv5
    
    
    end_conv = tf.identity(h_conv5 , name='identity_h_conv5')
    print 'end_conv' , end_conv
    #print conv2d(h_pool1 , w_conv2).get_shape()


Tensor("convolution_1/Relu:0", shape=(?, 16, 16, 200), dtype=float32, device=/device:GPU:0)
Tensor("convolution_2/conv2_maxpool:0", shape=(?, 8, 8, 200), dtype=float32, device=/device:GPU:0)
Tensor("convolution_3/Relu:0", shape=(?, 8, 8, 200), dtype=float32, device=/device:GPU:0)
Tensor("convolution_4/Relu:0", shape=(?, 8, 8, 200), dtype=float32, device=/device:GPU:0)
Tensor("convolution_5/conv5_maxpool:0", shape=(?, 4, 4, 200), dtype=float32, device=/device:GPU:0)
end_conv Tensor("identity_h_conv5:0", shape=(?, 4, 4, 200), dtype=float32, device=/device:GPU:0)


In [9]:
end_conv_row=int(h_conv5.get_shape()[1])
end_conv_col=int(h_conv5.get_shape()[2])
end_conv_ch=int(h_conv5.get_shape()[3])
#connect fully connected layer 
with tf.device('/gpu:0'):
    with tf.variable_scope("fc1") as scope:
        try:
            w_fc1=tf.get_variable("fc1_W",[end_conv_col*end_conv_row*end_conv_ch,fully_ch1] , initializer = tf.contrib.layers.xavier_initializer())
        except:
            scope.reuse_variables()
            w_fc1=tf.get_variable("fc1_W",[end_conv_col*end_conv_row*end_conv_ch,fully_ch1] , initializer = tf.contrib.layers.xavier_initializer())
        try:
            b_fc1 = bias_variable([fully_ch1] , name='fc1_B')
        except:
            scope.reuse_variables()
            b_fc1 = bias_variable([fully_ch1],'fc2_B')


    with tf.device('/gpu:0'): # flat conv layer 
        end_flat_conv =tf.reshape(end_conv, [-1,end_conv_col*end_conv_row*end_conv_ch] , name='endlayer_reshape')

    with tf.device('/gpu:0'): # connect flat layer with fully  connnected layer 
        h_fc1 = tf.nn.relu(tf.matmul(end_flat_conv , w_fc1 ,name='matmul_flat_fc1')+ b_fc1)
        h_fc1 = tf.nn.dropout(h_fc1, keep_prob , name='fc1_dropout')

In [10]:
with tf.device('/gpu:0'):
    with tf.variable_scope('fc2') as scope:
        try:
            w_fc2 =tf.get_variable("fc2_W",[fully_ch1 , fully_ch2],initializer = tf.contrib.layers.xavier_initializer())
        except:
            scope.reuse_variables()
            w_fc2 =tf.get_variable("fc2_W",[fully_ch1 , fully_ch2],initializer = tf.contrib.layers.xavier_initializer())
        try:
            b_fc2 = bias_variable([fully_ch2] , 'fc2_B')
        except:
            scope.reuse_variables()
            b_fc2 = bias_variable([fully_ch2], 'fc2_B')

    with tf.device('/gpu:0'):  # join flat layer with fully  connnected layer 
        h_fc2 = tf.nn.relu(tf.matmul(h_fc1 , w_fc2, name='matmul_fc1_fc2')+b_fc2 ,name= 'fc2_relu')
        h_fc2= tf.nn.dropout(h_fc2 , keep_prob , name='fc2_dropout')
    

In [11]:
with tf.device('/gpu:0'):
    with tf.variable_scope('fc3') as scope:
        try:
            w_fc3 =tf.get_variable("fc3_W",[fully_ch2 , fully_ch3],initializer = tf.contrib.layers.xavier_initializer())
        except:
            scope.reuse_variables()
            w_fc3 =tf.get_variable("fc3_W",[fully_ch2 , fully_ch3],initializer = tf.contrib.layers.xavier_initializer())
        try:
            b_fc3 = bias_variable([fully_ch3] ,name='fc3_B')
        except:
            scope.reuse_variables()
            b_fc3 = bias_variable([fully_ch3],'fc3_B')

    with tf.device('/gpu:0'):  # join flat layer with fully  connnected layer 
        h_fc3 = tf.nn.relu(tf.matmul(h_fc2 , w_fc3 , name='matmul_fc2_w_fc3')+b_fc3 , name='fc3_relu')
        h_fc3= tf.nn.dropout(h_fc3 , keep_prob , name='fc3_relu')
        end_fc=tf.identity(h_fc3 , name='identityh_fc3')

In [12]:
with tf.device('/gpu:0'):
    with tf.variable_scope('end_layer') as scope:
        try:
            w_end =tf.get_variable("end_W",[fully_ch3 , n_classes ],initializer = tf.contrib.layers.xavier_initializer())
        except:
            scope.reuse_variables()
            w_end =tf.get_variable("end_W",[fully_ch3 , n_classes],initializer = tf.contrib.layers.xavier_initializer())
        try:
            b_end = bias_variable([n_classes],name="end_B")
        except:
            scope.reuse_variables()
            b_end = bias_variable([n_classes],name="end_B")

    with tf.device('/gpu:0'):  # join flat layer with fully  connnected layer 
        y_conv = tf.matmul(h_fc3 , w_end , name="matmul_h_fc3_w_end")+b_end


In [13]:
#dirname = '/home/ncc/notebook/mammo/result/'

dirname='/home/user01/notebook/'
    
count=0
while(True):
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
        break
    elif not os.path.isdir(dirname + str(count)):
        dirname=dirname+str(count)
        os.mkdir(dirname)
        break
    else:
        count+=1
print 'it is recorded at :'+str(count)

it is recorded at :106


In [14]:
f=open(dirname+"/log.txt",'w')

In [15]:
def get_batch_list(folder_path):
    list_files=os.walk(folder_path).next()[2]
    print list_files
    ret_train_img_list=[]
    ret_train_lab_list=[]
    for i , ele in enumerate(list_files):

        if 'train'  in ele and 'img'in ele:
            ret_train_img_list.append(ele)
        elif 'train' in ele  and  'lab' in ele:
            ret_train_lab_list.append(ele)
    return ret_train_img_list ,ret_train_lab_list 

In [16]:
train_images , train_labels  = get_batch_list(file_locate)

['test_lab.npy', 'val_lab.npy', 'train_img.npy', 'test_img.npy', 'val_img.npy', 'train_lab.npy']


In [17]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]


train_images.sort(key=natural_keys)
train_labels.sort(key = natural_keys)
print(train_images)
print(train_labels)

['train_img.npy']
['train_lab.npy']


In [18]:
with tf.device('/gpu:0'):
#sm_conv= tf.nn.softmax(y_conv)
    #cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
    start_time = time.time()
with tf.name_scope("training"):
    regular=0.01*(tf.reduce_sum(tf.square(y_conv)))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( y_conv, y_ , name='softmax_cross_entropy') )
    with tf.device('/gpu:0'):
        cost = tf.add(cost,regular , name='add_cost_reg') 
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cost) #1e-4
        with tf.name_scope("accuracy"):
            with tf.name_scope('correct_prediction'):
                correct_prediction = tf.equal(tf.argmax(y_conv,1) ,tf.argmax(y_,1))
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(correct_prediction , "float")) 
tf.summary.scalar("accuracy " , accuracy)
sess = tf.Session()
merge = tf.merge_all_summaries()
writer= tf.train.SummaryWriter("./tensorboard3"+'/validataion_accuracy' ,sess.graph)
sess.run(tf.initialize_all_variables())

if divide_flag ==True:
    n_batch =len(train_images)
    batch_count=0

for i in range(iterate):    
    if divide_flag ==True:
        if batch_count >= n_batch:
            batch_count =0
        train_img =np.load(file_locate+train_images[batch_count])
    
        train_lab =np.load(file_locate+train_labels[batch_count])
    batch_xs , batch_ys = next_batch(batch_size, train_img , train_lab)
   # batch_val_xs  , batch_val_ys = next_batch(20 , val_img , val_lab)
    if i%100 ==0: # in here add to validation 
        try:
            result,val_accuracy = sess.run(merge,accuracy , feed_dict={x:val_img , y_:val_lab , keep_prob: 1.0})        
            val_loss = sess.run(cost , feed_dict = {x:val_img , y_: val_lab , keep_prob: 1.0})
            writer.add_summary(result, step)
            train_accuracy = sess.run( accuracy , feed_dict={x:batch_xs , y_:batch_ys , keep_prob: 1.0})        
            train_loss = sess.run(cost , feed_dict = {x:batch_xs, y_: batch_ys, keep_prob: 1.0})

            #result = sess.run(sm_conv , feed_dict = {x:val_img , y_:batch_ys , keep_prob :1.0})
            print("step %d , training  accuracy %g" %(i,train_accuracy))
            print("step %d , loss : %g" %(i,train_loss))
            train_str = 'step:\t'+str(i)+'\tval_loss:\t'+str(train_loss) +'\tval accuracy:\t'+str(train_accuracy)+'\n'
          
            print("step %d , validation  accuracy %g" %(i,val_accuracy))
            print("step %d , validation loss : %g" %(i,val_loss))
            val_str = 'step:\t'+str(i)+'\tval_loss:\t'+str(val_loss) +'\tval accuracy:\t'+str(val_accuracy)+'\n'
            
            
            f.write(val_str)
            f.write(train_str)
            if divide_flag ==True:
                batch_count+=1
        except :
            list_acc=[]
            list_loss=[]
            n_divide=len(val_img)/batch_size
            j=0
            for j in range(n_divide):
                
                # j*batch_size :(j+1)*batch_size
                val_accuracy,val_loss = sess.run([accuracy ,cost], feed_dict={x:val_img[ j*batch_size :(j+1)*batch_size] , y_:val_lab[ j*batch_size :(j+1)*batch_size ] , keep_prob: 1.0})        
                list_acc.append(float(val_accuracy))
                list_loss.append(float(val_loss))
            #right above code have to modify
            val_accuracy,val_loss = sess.run([accuracy ,cost], feed_dict={x:val_img[ j*batch_size :] , y_:val_lab[ j*batch_size :  ] , keep_prob: 1.0})         
            list_acc.append(val_accuracy)
            list_loss.append(val_loss)
            
            list_acc=np.asarray(list_acc)
            list_loss= np.asarray(list_loss)
            
            val_accuracy=np.mean(list_acc)
            val_loss = np.mean(list_loss)
            
            #result = sess.run(sm_conv , feed_dict = {x:val_img , y_:batch_ys , keep_prob :1.0})
            
            train_accuracy = sess.run( accuracy , feed_dict={x:batch_xs , y_:batch_ys , keep_prob: 1.0})        
            train_loss = sess.run(cost , feed_dict = {x:batch_xs, y_: batch_ys, keep_prob: 1.0})

            print("step %d , training  accuracy %g" %(i,train_accuracy))
            print("step %d , loss : %g" %(i,train_loss))
            train_str = 'step:\t'+str(i)+'\tval_loss:\t'+str(train_loss) +'\tval accuracy:\t'+str(train_accuracy)+'\n'
            
            print("step %d , validation  accuracy %g" %(i,val_accuracy))
            print("step %d , validation loss : %g" %(i,val_loss))
            val_str = 'step:\t'+str(i)+'\tval_loss:\t'+str(val_loss) +'\tval accuracy:\t'+str(val_accuracy)+'\n'
           
            
            f.write(val_str)
            f.write(train_str)
            if divide_flag ==True:
                batch_count+=1
    
    sess.run(train_step ,feed_dict={x:batch_xs , y_:batch_ys , keep_prob : 0.7})
    
print("--- Training Time : %s ---" % (time.time() - start_time))
train_time="--- Training Time : ---:\t" +str(time.time() - start_time)
f.write(train_time)

step 0 , training  accuracy 0.0666667
step 0 , loss : 35.0818
step 0 , validation  accuracy 0.0976447
step 0 , validation loss : 40.161
step 100 , training  accuracy 0.333333
step 100 , loss : 2.27469
step 100 , validation  accuracy 0.222675
step 100 , validation loss : 2.28392
step 200 , training  accuracy 0.1
step 200 , loss : 2.32305
step 200 , validation  accuracy 0.258483
step 200 , validation loss : 2.2681
step 300 , training  accuracy 0.233333
step 300 , loss : 2.27926
step 300 , validation  accuracy 0.26503
step 300 , validation loss : 2.26096
step 400 , training  accuracy 0.233333
step 400 , loss : 2.27232
step 400 , validation  accuracy 0.286627
step 400 , validation loss : 2.25178
step 500 , training  accuracy 0.2
step 500 , loss : 2.26219
step 500 , validation  accuracy 0.277605
step 500 , validation loss : 2.2542
step 600 , training  accuracy 0.233333
step 600 , loss : 2.25407
step 600 , validation  accuracy 0.276727
step 600 , validation loss : 2.25228
step 700 , training

In [19]:
for i in range(3):
    print ''
print i




2


In [20]:
try:
    test_accuracy = sess.run( accuracy , feed_dict={x:test_img , y_:test_lab , keep_prob: 1.0})        
    test_loss = sess.run(cost , feed_dict = {x:test_img , y_: test_lab , keep_prob: 1.0})

    #result = sess.run(sm_conv , feed_dict = {x:test_img , y_:batch_ys , keep_prob :1.0})
    print("step %d , testidation  accuracy %g" %(i,test_accuracy))
    print("step %d , testidation loss : %g" %(i,test_loss))
    test_str = 'step:\t'+str(i)+'\ttest_loss:\t'+str(test_loss) +'\ttest accuracy:\t'+str(test_accuracy)+'\n'

    f.write(test_str)
except :
    list_acc=[]
    list_loss=[]
    n_divide=len(test_img)/batch_size
    for j in range(n_divide):

        # j*batch_size :(j+1)*batch_size
        test_accuracy,test_loss = sess.run([accuracy ,cost], feed_dict={x:test_img[ j*batch_size :(j+1)*batch_size] , y_:test_lab[ j*batch_size :(j+1)*batch_size ] , keep_prob: 1.0})        
        list_acc.append(float(test_accuracy))
        list_loss.append(float(test_loss))
    test_accuracy , test_loss=sess.run([accuracy,cost] , feed_dict={x:test_img[(j+1)*batch_size : ] , y_:test_lab[(j+1)*(batch_size) : ] , keep_prob : 1.0})
    #right above code have to modify

    list_acc.append(test_accuracy)
    list_loss.append(test_loss)
    list_acc=np.asarray(list_acc)
    list_loss= np.asarray(list_loss)

    test_accuracy=np.mean(list_acc)
    test_loss = np.mean(list_loss)

    #result = sess.run(sm_conv , feed_dict = {x:test_img , y_:batch_ys , keep_prob :1.0})
    print("step %d , testidation  accuracy %g" %(i,test_accuracy))
    print("step %d , testidation loss : %g" %(i,test_loss))
    test_str = 'step:\t'+str(i)+'\ttest_loss:\t'+str(test_loss) +'\ttest accuracy:\t'+str(test_accuracy)+'\n'

    f.write(test_str)
    

step 2 , testidation  accuracy 0.3516
step 2 , testidation loss : 11.0528


In [21]:
sess.close()
